<a href="https://colab.research.google.com/github/wkd-woo/finance-statements/blob/main/%EC%9E%AC%EB%AC%B4%EC%9D%BC%EA%B4%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U finance-datareader

In [47]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [5]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [ ]:
print(재무상태표[:5])
print('------------------------------------------------------------------------------------------')
print(손익계산서[:5])
print('------------------------------------------------------------------------------------------')
print(포괄손익계산서[:5])
print('------------------------------------------------------------------------------------------')
print(현금흐름표[:5])
print('------------------------------------------------------------------------------------------')
print(자본변동표[:5])
print('------------------------------------------------------------------------------------------')

In [9]:
path = '/content/drive/MyDrive/22캡스톤디자인/재무일괄/2017_3분기보고서_01_재무상태표_연결_.txt'

In [57]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

In [58]:
for path in tqdm(재무상태표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기말' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기말' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기말' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 자산총계(ifrs_Assets)
  자산총계항목명 = df[df['항목명'].str.contains('자산 총계|자산총계')]
  자산총계항목코드 = df[(df['항목코드'] == 'ifrs_Assets') | (df['항목코드'] == 'ifrs-full_Assets')]
  항목명기준_diff = abs(전체종목 - len(자산총계항목명))
  항목코드기준_diff = abs(전체종목 - len(자산총계항목코드))

  if 항목코드기준_diff <= 항목명기준_diff:
    # 기준: 항목코드
    자산총계 = 자산총계항목코드  
  else:
    # 기준: 항목명
    자산총계 = 자산총계항목명

  자산총계.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(자산총계, sort=False, ignore_index=True)

  # 자본총계(ifrs_Equity)
  mask = df['항목명'].isin(['부채및자본총계', '부채와 자본총계', '부채와자본총계', '부채 및 자본총계', '지배주주자본 총계', '부채와 자본 총계', '부채 및 자본 총계', '부채와자본 총계', '부채과자본총계'])
  temp = df[~mask]

  자본총계항목명 = temp[temp['항목명'].str.contains('자본총계|자본 총계')].drop_duplicates(['종목코드', '보고서종류', '항목명']) # 기준: 항목명
  자본총계항목코드 = df[(df['항목코드'] == 'ifrs_Equity') | (df['항목코드'] == 'ifrs-full_Equity')]
  항목명기준_diff = abs(전체종목 - len(자본총계항목명))
  항목코드기준_diff = abs(전체종목 - len(자본총계항목코드))

  if 항목코드기준_diff <= 항목명기준_diff:
    # 기준: 항목코드
    자본총계 = 자본총계항목코드 
  else:
    # 기준: 항목명
    자본총계 = 자본총계항목명

  자본총계.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(자본총계, sort=False, ignore_index=True)

  # 부채총계(ifrs_Liabilities)
  mask = df['항목명'].isin(['부채및자본총계', '부채와 자본총계', '부채와자본총계', '부채 및 자본총계', '지배주주자본 총계', '부채와 자본 총계', '부채 및 자본 총계', '부채와자본 총계', '부채과자본총계'])
  temp = df[~mask]

  부채총계항목명 = temp[temp['항목명'].str.contains('부채총계|부채 총계')].drop_duplicates(['종목코드', '보고서종류', '항목명'])
  부채총계항목코드 = df[(df['항목코드'] == 'ifrs_Liabilities') | (df['항목코드'] == 'ifrs-full_Liabilities')]
  항목명기준_diff = abs(전체종목 - len(부채총계항목명))
  항목코드기준_diff = abs(전체종목 - len(부채총계항목코드))

  if 항목코드기준_diff <= 항목명기준_diff:
    # 기준: 항목코드
    부채총계 = 부채총계항목코드
  else:
    # 기준: 항목명
    부채총계 = 부채총계항목명

  부채총계.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(부채총계, sort=False, ignore_index=True)

  # 유동자산(ifrs_CurrentAssets)
  mask = df['항목명'].str.contains('비유동|비 유동|기타|매각|합계')
  temp = df[~mask]
  
  유동자산항목명 = temp[temp['항목명'].str.contains('유동자산|유동 자산')].drop_duplicates(['종목코드', '보고서종류', '항목명'])
  유동자산항목코드 = df[(df['항목코드'] == 'ifrs_CurrentAssets') | (df['항목코드'] == 'ifrs-full_CurrentAssets')]
  항목명기준_diff = abs(전체종목 -  len(유동자산항목명))
  항목코드기준_diff = abs(전체종목 - len(유동자산항목코드))

  if 항목코드기준_diff <= 항목명기준_diff:
    # 기준: 항목코드
    유동자산 = 유동자산항목코드
  else:
    # 기준: 항목명
    유동자산 = 유동자산항목명
  
  유동자산.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(유동자산, sort=False, ignore_index=True)

  # 현금 및 현금성자산(ifrs_CashAndCashEquivalents)
  mask = df['항목명'].str.contains('국고보조금(현금차감)|현금흐름위험회피파생상품평가손실')
  temp = df[~mask]
  
  현금성자산항목명 = temp[temp['항목명'].str.contains('현금')].drop_duplicates(['종목코드', '보고서종류', 당기])
  현금성자산항목코드 = df[(df['항목코드'] == 'ifrs_CashAndCashEquivalents') | (df['항목코드'] == 'ifrs-full_CashAndCashEquivalents')]
  항목명기준_diff = abs(전체종목 -  len(현금성자산항목명))
  항목코드기준_diff = abs(전체종목 - len(현금성자산항목코드))

  if 항목코드기준_diff <= 항목명기준_diff:
    # 기준: 항목코드
    현금성자산 = 현금성자산항목코드
  else:
    # 기준: 항목명
    현금성자산 = 현금성자산항목명

  현금성자산.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(현금성자산, sort=False, ignore_index=True)


  0%|          | 0/24 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
 21%|██        | 5/24 [00:04<00:17,  1.09it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 24/24 [00:24<00:00,  1.01s/it]


In [60]:
sample['항목명'].unique()

array(['자산총계', '자산 총계', '자    산    총    계', '자  산  총  계', '자본총계', '당분기말',
       '분기말자본', '기말', '분기말', '반기말자본', '기말 자본', 'IV. 기말자본', '자본 총계',
       '분기말 잔액', '분기말잔액', '자    본    총    계', '당반기말', '자기주식의 취득',
       '3분기말자본', '분기말 자본', '부채총계', '부채 총계', '부    채    총    계',
       '부  채  총  계', '유동자산', 'Ⅰ.유동자산', 'I. 유동자산', 'l. 유동자산', 'Ⅰ. 유동자산',
       'I.유동자산', '현금및현금성자산', '현금및예치금', '현금 및 현금성자산', '(1)현금및현금성자산',
       '1.현금및현금성자산', '1. 현금및현금성자산', '자본에 직접 반영된 소유주와의 거래', '기말잔액', '당기말',
       'IV.당(전)기말', '기말금액', '자  본  총  계', '당기말자본', '자  본  계',
       '2015.12.31 기말자본', '부  채  계', 'II. 유동자산', '유동 자산', '1)현금및현금성자산',
       '자 본 총 계', '부 채 총 계', 'II.유동자산', '1. 현금 및 현금성자산', 'IV.기말자본',
       '자본합계', '(당기말)', '자본계', '1분기말자본', '(당분기말)', '당분기말 자본',
       '2018.9.30(당분기말)', '반기말', '기말자본', '당분기말자본', '파생금융유동자산', '(당반기말)',
       '당반기말자본', '전기말', '총계', '2019.03.31(당분기말)', '당분기말(2019년9월30일)잔액',
       '전분기말자본', '반기말 자본', '비금융유동자산', '당기말(2019년12월31일)잔액', '기말자본 잔액',
       '당분기말(2020년3월31일)잔액', '당분기말

In [62]:
재무일괄 = sample
재무일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,3분기보고서,3S,262,ifrs-full_Assets,자산총계,"65,694,406,823"
1,[095570],2021,3분기보고서,AJ네트웍스,763,ifrs-full_Assets,자산총계,"1,357,720,740,483"
2,[006840],2021,3분기보고서,AK홀딩스,649,ifrs-full_Assets,자산총계,"4,361,681,811,190"
3,[054620],2021,3분기보고서,APS홀딩스,649,ifrs-full_Assets,자산총계,"287,247,655,273"
4,[265520],2021,3분기보고서,AP시스템,292,ifrs-full_Assets,자산총계,"445,457,356,474"
...,...,...,...,...,...,...,...,...
190913,[069260],2016,반기보고서,휴켐스,204,ifrs_CashAndCashEquivalents,현금및현금성자산,"38,326,395,822"
190914,[010240],2016,반기보고서,흥국,292,ifrs_CashAndCashEquivalents,현금및현금성자산,"2,655,098,424"
190915,[189980],2016,반기보고서,흥국에프엔비,112,ifrs_CashAndCashEquivalents,현금및현금성자산,"1,107,330,416"
190916,[003280],2016,반기보고서,흥아해운,501,ifrs_CashAndCashEquivalents,현금및현금성자산,"20,664,686,060"


In [66]:
len(재무일괄['종목코드'].unique())

1996

In [68]:
재무일괄['사업연도'].unique()

array(['2021', '2015', '2016', '2017', '2018', '2019', '2020'],
      dtype=object)

```javascript
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)
```

In [69]:
재무일괄[재무일괄['항목코드'] == 'entity00632304_udf_BS_2013325173927154_StatementOfFinancialPositionAbstract']

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
9007,[096760],2015,사업보고서,JW홀딩스,715,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,082,862,852,768"
15873,[096760],2016,사업보고서,JW홀딩스,715,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,196,629,708,762"
23164,[096760],2017,1분기보고서,JW홀딩스,715,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,277,421,495,623"
37981,[096760],2017,반기보고서,JW홀딩스,649,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,290,077,515,365"
170000,[096760],2016,1분기보고서,JW홀딩스,715,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,115,904,764,705"
176893,[096760],2016,3분기보고서,JW홀딩스,715,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,148,488,597,153"
183973,[096760],2016,반기보고서,JW홀딩스,715,entity00632304_udf_BS_2013325173927154_Stateme...,자산총계,"1,120,722,901,718"


In [72]:
재무일괄[재무일괄['항목코드'] == 'dart_CurrentDerivativeAsset']

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
66174,[003230],2018,3분기보고서,삼양식품,107,dart_CurrentDerivativeAsset,파생금융유동자산,"317,378,891"
74061,[003230],2018,반기보고서,삼양식품,107,dart_CurrentDerivativeAsset,파생금융유동자산,"344,940,027"
115012,[003230],2019,사업보고서,삼양식품,107,dart_CurrentDerivativeAsset,파생금융유동자산,NaN


In [70]:
재무일괄['항목코드'].unique()

array(['ifrs-full_Assets', 'ifrs-full_Equity', 'ifrs-full_Liabilities',
       'ifrs-full_CurrentAssets', 'ifrs-full_CashAndCashEquivalents',
       'ifrs_Assets',
       'entity00632304_udf_BS_2013325173927154_StatementOfFinancialPositionAbstract',
       'entity00120562_udf_BS_20138910391246_StatementOfFinancialPositionAbstract',
       'entity00126308_udf_BS_201152920245946_StatementOfFinancialPositionAbstract',
       'entity00300405_udf_BS_20163315650101_StatementOfFinancialPositionAbstract',
       'entity00526696_udf_BS_2013522183339953_AssetsAbstract',
       'entity00361169_udf_BS_201451320532358_udf_BS_201451320056207_StatementOfFinancialPositionAbstract',
       'entity00306454_udf_BS_2014328191236116_StatementOfFinancialPositionAbstract',
       'entity00161693_udf_BS_2015318112454167_AssetsAbstract',
       'entity00300548_udf_BS_2014322121322670_StatementOfFinancialPositionAbstract',
       'ifrs_Equity', 'ifrs_Liabilities', 'ifrs_CurrentAssets',
       'entity00112378_ud